In [1]:
from functools import partial

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 30)

from server2022.data.processed.prepare_data import create_df_2years_known, create_df_0years_known, create_df_1year_known
from server2022.data.processed.prepare_data import factors_2020, FINANCE_FEAT

In [2]:
df_1 = create_df_1year_known()

In [3]:
df_1

,Наименование ДП,Факт. 1,Факт. 2,Факт. 3,Факт. 7,Факт. 12,Факт. 14,Факт. 15,Факт. 16,Факт. 20,Факт. 21,Факт. 23,Факт. 24,Факт. 27,Факт. 28,Факт. 29,Факт. 30,Факт. 31,Факт. 32,Факт. 33,Факт. 37,Факт. 39,Факт. 40,Факт. 41,Факт. 42,Факт. 46,Факт. 48,Факт. 49,Факт. 50,Факт. 51,Факт. 54,Факт. 55,Факт. 56,Факт. 57,Факт. 58,Факт. 59,Факт. 60,Итого,"-4, Нематериальные активы, RUB","-3, Нематериальные активы, RUB","-2, Нематериальные активы, RUB","-1, Нематериальные активы, RUB","-4, Основные средства , RUB","-3, Основные средства , RUB","-2, Основные средства , RUB","-1, Основные средства , RUB","-4, Внеоборотные активы, RUB","-3, Внеоборотные активы, RUB","-2, Внеоборотные активы, RUB","-1, Внеоборотные активы, RUB","-4, Дебиторская задолженность, RUB","-3, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB","-1, Дебиторская задолженность, RUB","-4, Оборотные активы, RUB","-3, Оборотные активы, RUB","-2, Оборотные активы, RUB","-1, Оборотные активы, RUB","-4, Уставный капитал , RUB","-3, Уставный капитал , RUB","-2, Уставный капитал , RUB","-1, Уставный капитал , RUB","-4, Капитал и резервы, RUB","-3, Капитал и резервы, RUB","-2, Капитал и резервы, RUB","-1, Капитал и резервы, RUB","-4, Заёмные средства (долгосрочные), RUB","-3, Заёмные средства (долгосрочные), RUB","-2, Заёмные средства (долгосрочные), RUB","-1, Заёмные средства (долгосрочные), RUB","-4, Долгосрочные обязательства, RUB","-3, Долгосрочные обязательства, RUB","-2, Долгосрочные обязательства, RUB","-1, Долгосрочные обязательства, RUB","-4, Заёмные средства (краткосрочные), RUB","-3, Заёмные средства (краткосрочные), RUB","-2, Заёмные средства (краткосрочные), RUB","-1, Заёмные средства (краткосрочные), RUB","-4, Кредиторская задолженность, RUB","-3, Кредиторская задолженность, RUB","-2, Кредиторская задолженность, RUB","-1, Кредиторская задолженность, RUB","-4, Краткосрочные обязательства, RUB","-3, Краткосрочные обязательства, RUB","-2, Краткосрочные обязательства, RUB","-1, Краткосрочные обязательства, RUB","-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB","-4, Себестоимость продаж, RUB","-3, Себестоимость продаж, RUB","-2, Себестоимость продаж, RUB","-1, Себестоимость продаж, RUB","-4, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","-4, Прибыль (убыток) от продажи, RUB","-3, Прибыль (убыток) от продажи, RUB",...,"Normalized -2, Уставный капитал , RUB","Normalized -1, Уставный капитал , RUB","Normalized -4, Капитал и резервы, RUB","Normalized -3, Капитал и резервы, RUB","Normalized -2, Капитал и резервы, RUB","Normalized -1, Капитал и резервы, RUB","Normalized -4, Долгосрочные обязательства, RUB","Normalized -3, Долгосрочные обязательства, RUB","Normalized -2, Долгосрочные обязательства, RUB","Normalized -1, Долгосрочные обязательства, RUB","Normalized -2, Заёмные средства (краткосрочные), RUB","Normalized -1, Заёмные средства (краткосрочные), RUB","Normalized -4, Кредиторская задолженность, RUB","Normalized -3, Кредиторская задолженность, RUB","Normalized -2, Кредиторская задолженность, RUB","Normalized -1, Кредиторская задолженность, RUB","Normalized -4, Краткосрочные обязательства, RUB","Normalized -3, Краткосрочные обязательства, RUB","Normalized -2, Краткосрочные обязательства, RUB","Normalized -1, Краткосрочные обязательства, RUB","Normalized -4, Выручка, RUB","Normalized -3, Выручка, RUB","Normalized -2, Выручка, RUB","Normalized -1, Выручка, RUB","Normalized -4, Себестоимость продаж, RUB","Normalized -3, Себестоимость продаж, RUB","Normalized -2, Себестоимость продаж, RUB","Normalized -1, Себестоимость продаж, RUB","Normalized -4, Прибыль (убыток) до налогообложения , RUB","Normalized -3, Прибыль (убыток) до налогообложения , RUB","Normalized -2, Прибыль (убыток) до налогообложения , RUB","Normalized -1, Прибыль (убыток) до налогообложения , RUB","Normalized -4, Прибыль (убыток) 

In [8]:
len(df_0[df_0['year'] == '2020'])

531

In [7]:
df_0[df_0['year'] == '2020']['binary_target'].sum()

363

In [35]:
col_name

'-2, Дебиторская задолженность, RUB'

In [45]:
df = df_1.copy()
#def normalize_according2year()
index_name = 'Наименование ДП'
df_2020 = df[df['year'] == '2020'].set_index(index_name)[[col_name]]
df_2021 = df[df['year'] == '2021'].set_index(index_name)[[col_name]]

joined = df_2021.join(df_2020, rsuffix=' prev')

def func(row):
    if (np.abs(row[col_name]) < 10) or (np.abs(row[col_name + ' prev']) < 10):
        return np.nan
    
    return row[col_name + ' prev'] / row[col_name]
    
multiplier = joined.apply(func, axis=1).median(skipna=True)

df.loc[df['year'] == '2020', 'Normalized ' + col_name] = df.loc[df['year'] == '2020', col_name]
df.loc[df['year'] == '2021', 'Normalized ' + col_name] = df.loc[df['year'] == '2021', col_name] * multiplier

In [47]:
df[[col_name, 'Normalized ' + col_name, 'year']]

,"-2, Дебиторская задолженность, RUB","Normalized -2, Дебиторская задолженность, RUB",year
0,3.475461e+09,3.475461e+09,2020
1,1.563478e+08,1.563478e+08,2020
2,2.280497e+08,2.280497e+08,2020
3,5.362190e+09,5.362190e+09,2020
4,1.124670e+10,1.124670e+10,2020
...,...,...,...
851,3.080000e+07,2.703869e+07,2021
852,0.000000e+00,0.000000e+00,2021
853,2.847134e+07,2.499440e+07,2021
854,1.457293e+08,1.279328e+08,2021


In [31]:
joined['multiplier'].median()

0.8778794700876283

In [29]:
joined

,"-2, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB prev",multiplier
Наименование ДП,,,
0,3.692142e+09,3.475461e+09,0.941313
2,2.770312e+08,2.280497e+08,0.823191
4,1.052114e+10,1.124670e+10,1.068962
5,1.902017e+10,6.165455e+09,0.324154
6,8.921848e+09,1.099587e+10,1.232466
...,...,...,...
522,3.080000e+07,0.000000e+00,NaN
523,0.000000e+00,0.000000e+00,NaN
525,2.847134e+07,0.000000e+00,NaN


In [21]:
df_2020

,"-2, Дебиторская задолженность, RUB"
Наименование ДП,
0,3.475461e+09
1,1.563478e+08
2,2.280497e+08
3,5.362190e+09
4,1.124670e+10
...,...
526,2.566242e+07
527,8.277452e+07
528,0.000000e+00


In [4]:
df_0 = create_df_0years_known()

In [5]:
df_0

,Наименование ДП,"-3, Нематериальные активы, RUB","-2, Нематериальные активы, RUB","-1, Нематериальные активы, RUB","-3, Основные средства , RUB","-2, Основные средства , RUB","-1, Основные средства , RUB","-3, Внеоборотные активы, RUB","-2, Внеоборотные активы, RUB","-1, Внеоборотные активы, RUB","-3, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB","-1, Дебиторская задолженность, RUB","-3, Оборотные активы, RUB","-2, Оборотные активы, RUB","-1, Оборотные активы, RUB","-3, Уставный капитал , RUB","-2, Уставный капитал , RUB","-1, Уставный капитал , RUB","-3, Капитал и резервы, RUB","-2, Капитал и резервы, RUB","-1, Капитал и резервы, RUB","-3, Заёмные средства (долгосрочные), RUB","-2, Заёмные средства (долгосрочные), RUB","-1, Заёмные средства (долгосрочные), RUB","-3, Долгосрочные обязательства, RUB","-2, Долгосрочные обязательства, RUB","-1, Долгосрочные обязательства, RUB","-3, Заёмные средства (краткосрочные), RUB","-2, Заёмные средства (краткосрочные), RUB","-1, Заёмные средства (краткосрочные), RUB","-3, Кредиторская задолженность, RUB","-2, Кредиторская задолженность, RUB","-1, Кредиторская задолженность, RUB","-3, Краткосрочные обязательства, RUB","-2, Краткосрочные обязательства, RUB","-1, Краткосрочные обязательства, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB","-3, Себестоимость продаж, RUB","-2, Себестоимость продаж, RUB","-1, Себестоимость продаж, RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) от продажи, RUB","-2, Прибыль (убыток) от продажи, RUB","-1, Прибыль (убыток) от продажи, RUB",year,binary_target,Итого,"-4, Нематериальные активы, RUB","-4, Основные средства , RUB","-4, Внеоборотные активы, RUB","-4, Дебиторская задолженность, RUB","-4, Оборотные активы, RUB","-4, Уставный капитал , RUB","-4, Капитал и резервы, RUB","-4, Заёмные средства (долгосрочные), RUB","-4, Долгосрочные обязательства, RUB","-4, Заёмные средства (краткосрочные), RUB","-4, Кредиторская задолженность, RUB","-4, Краткосрочные обязательства, RUB","-4, Выручка, RUB","-4, Себестоимость продаж, RUB","-4, Прибыль (убыток) до налогообложения , RUB","-4, Прибыль (убыток) от продажи, RUB",Нематериальные активы total mean growth,Основные средства total mean growth,Внеоборотные активы total mean growth,Дебиторская задолженность total mean growth,Оборотные активы total mean growth,Уставный капитал total mean growth,Капитал и резервы total mean growth,Заёмные средства (долгосрочные) total mean growth,Долгосрочные обязательства total mean growth,Заёмные средства (краткосрочные) total mean growth,Кредиторская задолженность total mean growth,Краткосрочные обязательства total mean growth,Выручка total mean growth,Себестоимость продаж total mean growth,Прибыль (убыток) до налогообложения total mean growth,Прибыль (убыток) от продажи total mean growth,"log -4, Нематериальные активы, RUB","log -3, Нематериальные активы, RUB","log -2, Нематериальные активы, RUB","log -1, Нематериальные активы, RUB","log -4, Основные средства , RUB","log -3, Основные средства , RUB","log -2, Основные средства , RUB","log -1, Основные средства , RUB","log -4, Внеоборотные активы, RUB","log -3, Внеоборотные активы, RUB","log -2, Внеоборотные активы, RUB","log -1, Внеоборотные активы, RUB","log -4, Дебиторская задолженность, RUB","log -3, Дебиторская задолженность, RUB","log -2, Дебиторская задолженность, RUB","log -1, Дебиторская задолженность, RUB","log -4, Оборотные активы, RUB","log -3, Оборотные активы, RUB","log -2, Оборотные активы, RUB","log -1, Оборотные активы, RUB","log -4, Уставный капитал , RUB","log -3, Уставный капитал , RUB","log -2, Уставный капитал , RUB","log -1, Уставный капитал , RUB","log -4, Капитал и резервы, RUB","log -3, Капитал и резервы, RUB","log -2, Капитал и резервы, RUB","log -1, Капитал и резервы, RUB","log -4, Заёмные средства (долгосрочные), RUB","log -3, Заёмные средства (долгосрочн

In [2]:
df_1 = create_df_1year_known()

In [3]:
df_1

,Наименование ДП,Факт. 1,Факт. 2,Факт. 3,Факт. 7,Факт. 12,Факт. 14,Факт. 15,Факт. 16,Факт. 20,Факт. 21,Факт. 23,Факт. 24,Факт. 27,Факт. 28,Факт. 29,Факт. 30,Факт. 31,Факт. 32,Факт. 33,Факт. 37,Факт. 39,Факт. 40,Факт. 41,Факт. 42,Факт. 46,Факт. 48,Факт. 49,Факт. 50,Факт. 51,Факт. 54,Факт. 55,Факт. 56,Факт. 57,Факт. 58,Факт. 59,Факт. 60,Итого,"-4, Нематериальные активы, RUB","-3, Нематериальные активы, RUB","-2, Нематериальные активы, RUB","-1, Нематериальные активы, RUB","-4, Основные средства , RUB","-3, Основные средства , RUB","-2, Основные средства , RUB","-1, Основные средства , RUB","-4, Внеоборотные активы, RUB","-3, Внеоборотные активы, RUB","-2, Внеоборотные активы, RUB","-1, Внеоборотные активы, RUB","-4, Дебиторская задолженность, RUB","-3, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB","-1, Дебиторская задолженность, RUB","-4, Оборотные активы, RUB","-3, Оборотные активы, RUB","-2, Оборотные активы, RUB","-1, Оборотные активы, RUB","-4, Уставный капитал , RUB","-3, Уставный капитал , RUB","-2, Уставный капитал , RUB","-1, Уставный капитал , RUB","-4, Капитал и резервы, RUB","-3, Капитал и резервы, RUB","-2, Капитал и резервы, RUB","-1, Капитал и резервы, RUB","-4, Заёмные средства (долгосрочные), RUB","-3, Заёмные средства (долгосрочные), RUB","-2, Заёмные средства (долгосрочные), RUB","-1, Заёмные средства (долгосрочные), RUB","-4, Долгосрочные обязательства, RUB","-3, Долгосрочные обязательства, RUB","-2, Долгосрочные обязательства, RUB","-1, Долгосрочные обязательства, RUB","-4, Заёмные средства (краткосрочные), RUB","-3, Заёмные средства (краткосрочные), RUB","-2, Заёмные средства (краткосрочные), RUB","-1, Заёмные средства (краткосрочные), RUB","-4, Кредиторская задолженность, RUB","-3, Кредиторская задолженность, RUB","-2, Кредиторская задолженность, RUB","-1, Кредиторская задолженность, RUB","-4, Краткосрочные обязательства, RUB","-3, Краткосрочные обязательства, RUB","-2, Краткосрочные обязательства, RUB","-1, Краткосрочные обязательства, RUB","-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB","-4, Себестоимость продаж, RUB","-3, Себестоимость продаж, RUB","-2, Себестоимость продаж, RUB","-1, Себестоимость продаж, RUB","-4, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","-4, Прибыль (убыток) от продажи, RUB","-3, Прибыль (убыток) от продажи, RUB","-2, Прибыль (убыток) от продажи, RUB","-1, Прибыль (убыток) от продажи, RUB",year,binary_target,"Макс. ПДЗ за -1 год, дней","Сред. ПДЗ за -1 год, дней","Кол-во просрочек свыше 5-ти дней за -1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за -1 год, руб.","Кол-во раз ПДЗ за -1 год, шт.",Нематериальные активы total mean growth,Основные средства total mean growth,Внеоборотные активы total mean growth,Дебиторская задолженность total mean growth,Оборотные активы total mean growth,Уставный капитал total mean growth,Капитал и резервы total mean growth,Заёмные средства (долгосрочные) total mean growth,Долгосрочные обязательства total mean growth,Заёмные средства (краткосрочные) total mean growth,Кредиторская задолженность total mean growth,Краткосрочные обязательства total mean growth,Выручка total mean growth,Себестоимость продаж total mean growth,Прибыль (убыток) до налогообложения total mean growth,Прибыль (убыток) от продажи total mean growth,"log -4, Нематериальные активы, RUB","log -3, Нематериальные активы, RUB","log -2, Нематериальные активы, RUB","log -1, Нематериальные активы, RUB","log -4, Основные средства , RUB","log -3, Основные средства , RUB","log -2, Основные средства , RUB","log -1, Основные средства , RUB","log -4, Внеоборотные активы, RUB","log -3, Внеоборотные активы, RUB","log -2, Внеоборотные активы, RUB","log -1, Внеоборотные активы, RUB","log -4, Дебиторская задолженность, RUB","log -3, Дебиторская задолженность, RUB","log -2, Дебиторская задолженность, RUB","log -1, 

In [7]:
df_1[df_1['-1, Выручка, RUB'] < 10]

,Наименование ДП,Факт. 1,Факт. 2,Факт. 3,Факт. 7,Факт. 12,Факт. 14,Факт. 15,Факт. 16,Факт. 20,Факт. 21,Факт. 23,Факт. 24,Факт. 27,Факт. 28,Факт. 29,Факт. 30,Факт. 31,Факт. 32,Факт. 33,Факт. 37,Факт. 39,Факт. 40,Факт. 41,Факт. 42,Факт. 46,Факт. 48,Факт. 49,Факт. 50,Факт. 51,Факт. 54,Факт. 55,Факт. 56,Факт. 57,Факт. 58,Факт. 59,Факт. 60,Итого,"-4, Нематериальные активы, RUB","-3, Нематериальные активы, RUB","-2, Нематериальные активы, RUB","-1, Нематериальные активы, RUB","-4, Основные средства , RUB","-3, Основные средства , RUB","-2, Основные средства , RUB","-1, Основные средства , RUB","-4, Внеоборотные активы, RUB","-3, Внеоборотные активы, RUB","-2, Внеоборотные активы, RUB","-1, Внеоборотные активы, RUB","-4, Дебиторская задолженность, RUB","-3, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB","-1, Дебиторская задолженность, RUB","-4, Оборотные активы, RUB","-3, Оборотные активы, RUB","-2, Оборотные активы, RUB","-1, Оборотные активы, RUB","-4, Уставный капитал , RUB","-3, Уставный капитал , RUB","-2, Уставный капитал , RUB","-1, Уставный капитал , RUB","-4, Капитал и резервы, RUB","-3, Капитал и резервы, RUB","-2, Капитал и резервы, RUB","-1, Капитал и резервы, RUB","-4, Заёмные средства (долгосрочные), RUB","-3, Заёмные средства (долгосрочные), RUB","-2, Заёмные средства (долгосрочные), RUB","-1, Заёмные средства (долгосрочные), RUB","-4, Долгосрочные обязательства, RUB","-3, Долгосрочные обязательства, RUB","-2, Долгосрочные обязательства, RUB","-1, Долгосрочные обязательства, RUB","-4, Заёмные средства (краткосрочные), RUB","-3, Заёмные средства (краткосрочные), RUB","-2, Заёмные средства (краткосрочные), RUB","-1, Заёмные средства (краткосрочные), RUB","-4, Кредиторская задолженность, RUB","-3, Кредиторская задолженность, RUB","-2, Кредиторская задолженность, RUB","-1, Кредиторская задолженность, RUB","-4, Краткосрочные обязательства, RUB","-3, Краткосрочные обязательства, RUB","-2, Краткосрочные обязательства, RUB","-1, Краткосрочные обязательства, RUB","-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB","-4, Себестоимость продаж, RUB","-3, Себестоимость продаж, RUB","-2, Себестоимость продаж, RUB","-1, Себестоимость продаж, RUB","-4, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","-4, Прибыль (убыток) от продажи, RUB","-3, Прибыль (убыток) от продажи, RUB","-2, Прибыль (убыток) от продажи, RUB","-1, Прибыль (убыток) от продажи, RUB",year,binary_target,"Макс. ПДЗ за -1 год, дней","Сред. ПДЗ за -1 год, дней","Кол-во просрочек свыше 5-ти дней за -1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за -1 год, руб.","Кол-во раз ПДЗ за -1 год, шт.",Нематериальные активы total mean growth,Основные средства total mean growth,Внеоборотные активы total mean growth,Дебиторская задолженность total mean growth,Оборотные активы total mean growth,Уставный капитал total mean growth,Капитал и резервы total mean growth,Заёмные средства (долгосрочные) total mean growth,Долгосрочные обязательства total mean growth,Заёмные средства (краткосрочные) total mean growth,Кредиторская задолженность total mean growth,Краткосрочные обязательства total mean growth,Выручка total mean growth,Себестоимость продаж total mean growth,Прибыль (убыток) до налогообложения total mean growth,Прибыль (убыток) от продажи total mean growth
52,52,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,10,Х,10,10,Х,Х,Н/Д,Н/Д,Х,Х,30.0,7.910828e+05,6.394904e+05,5.675159e+05,0.0,5.082541e+08,6.461707e+08,6.530662e+08,0.000000e+00,7.417382e+08,8.402911e+08,9.756395e+08,0.000000e+00,8.546459e+08,8.802076e+08,9.350191e+08,0.000000e+00,1.859778e+09,1.649237e+09,1.826028e+09,0.000000e+00,1.769427e+07,1.769427e+07,1.769427e+07,0.000000e+00,1.367961e+09,1.543850e+09,1.207076e+09,0.000000e+00,3.225930e+08,2.656051e+08,5.732484e+08,0.000000e+00,3.807051e+08,3.316866e+08,6.520229e+08,0.000000e+00,1.57926

In [9]:
(df_1['-1, Выручка, RUB'] < 10).sum()

20

In [10]:
list(range(-5,0))

[-5, -4, -3, -2, -1]

In [23]:
for col in FINANCE_FEAT:
    

SyntaxError: invalid syntax (<ipython-input-23-bf7fb86468f6>, line 1)

In [10]:
fin_feat = FINANCE_FEAT[3]
fin_feat

'Дебиторская задолженность'

In [11]:
some_cols = ['Наименование ДП', *[str(i) + ', ' + fin_feat + ', RUB' for i in range(-4, 0)]]
some_cols

['Наименование ДП',
 '-4, Дебиторская задолженность, RUB',
 '-3, Дебиторская задолженность, RUB',
 '-2, Дебиторская задолженность, RUB',
 '-1, Дебиторская задолженность, RUB']

In [12]:
col_name = some_cols[3]

In [102]:
df = df_1[some_cols]
df.head()

,Наименование ДП,"-4, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB"
0,0,3.179697e+09,6.167667e+09,7.461828e+08,3.603784e+09
1,1,1.212357e+07,6.944586e+06,4.480064e+07,8.747516e+07
2,2,5.026662e+08,3.644229e+08,8.552994e+07,-6.456439e+08
3,3,4.100857e+09,4.481281e+09,6.840634e+09,3.999298e+09
4,4,7.163980e+10,3.799862e+10,4.593606e+10,4.960408e+10


In [103]:
df[col_name] + 100_000_000

KeyError: '-1, Прибыль (убыток) от продажи, RUB'

In [77]:
q = df[col_name]

In [104]:
def count_log_values(column_values):
    minus_mask = column_values < 0
    zeros_mask = np.abs(column_values) < 0.1
    res = np.log(np.abs(column_values) + 1)
    res[minus_mask] *= -1
    res[zeros_mask] = np.median(res[~zeros_mask])
    return res

In [105]:
for fin_feat in FINANCE_FEAT:
    for i in range(-5, 0):
        col_name = str(i) + ', ' + fin_feat + ', RUB'
        if not col_name in df.columns.tolist():
            continue
        df['log ' + col_name] = count_log_values(df[col_name].values)

/home/zakharov_a/new_miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
df

,Наименование ДП,"-4, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","log -4, Прибыль (убыток) до налогообложения , RUB","log -3, Прибыль (убыток) до налогообложения , RUB","log -2, Прибыль (убыток) до налогообложения , RUB","log -1, Прибыль (убыток) до налогообложения , RUB"
0,0,3.179697e+09,6.167667e+09,7.461828e+08,3.603784e+09,21.880052,22.542586,20.430481,22.005250
1,1,1.212357e+07,6.944586e+06,4.480064e+07,8.747516e+07,16.310662,15.753473,17.617733,18.286865
2,2,5.026662e+08,3.644229e+08,8.552994e+07,-6.456439e+08,20.035437,19.713826,18.264377,-20.285759
3,3,4.100857e+09,4.481281e+09,6.840634e+09,3.999298e+09,22.134462,22.223175,22.646146,22.109385
4,4,7.163980e+10,3.799862e+10,4.593606e+10,4.960408e+10,24.994917,24.360816,24.550516,24.627339
...,...,...,...,...,...,...,...,...,...
851,522,0.000000e+00,0.000000e+00,2.569427e+06,4.008280e+06,16.006440,15.983908,14.759194,15.203873
852,523,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,16.006440,15.983908,16.083088,16.154250
853,525,0.000000e+00,0.000000e+00,3.095541e+05,4.284076e+06,16.006440,15.983908,12.642892,15.270416
854,527,1.553159e+08,8.043376e+07,6.917389e+07,2.835159e+07,18.860972,18.202945,18.052134,17.160194


In [107]:
np.log(3_999_000_000)

22.109310166811092

In [109]:
df['log -1, Прибыль (убыток) до налогообложения , RUB'].median()

16.154249537958098

In [108]:
np.log(645_000_000)

20.284760874760046

In [27]:
for i in range(-5, 0):
    col_name = str(i) + ', ' + fin_feat + ', RUB'
    if not col_name in df.columns.tolist():
        continue
    minus_mask = df[col_name] < 0
    df['log '+ col_name] = (df[col_name].abs() + 1).lo
    df.loc[minus_mask, 'log '+ col_name] *=

In [56]:
fin_feat = FINANCE_FEAT[14]
col_name = str(i) + ', ' + fin_feat + ', RUB'
print(col_name)
df_1[col_name].min(skipna=True)

-1, Прибыль (убыток) до налогообложения , RUB


-282967444585.9872

In [62]:
minus_mask = df_1[col_name] < 0

In [64]:
df_1[minus_mask]

,Наименование ДП,Факт. 1,Факт. 2,Факт. 3,Факт. 7,Факт. 12,Факт. 14,Факт. 15,Факт. 16,Факт. 20,Факт. 21,Факт. 23,Факт. 24,Факт. 27,Факт. 28,Факт. 29,Факт. 30,Факт. 31,Факт. 32,Факт. 33,Факт. 37,Факт. 39,Факт. 40,Факт. 41,Факт. 42,Факт. 46,Факт. 48,Факт. 49,Факт. 50,Факт. 51,Факт. 54,Факт. 55,Факт. 56,Факт. 57,Факт. 58,Факт. 59,Факт. 60,Итого,"-4, Нематериальные активы, RUB","-3, Нематериальные активы, RUB","-2, Нематериальные активы, RUB","-1, Нематериальные активы, RUB","-4, Основные средства , RUB","-3, Основные средства , RUB","-2, Основные средства , RUB","-1, Основные средства , RUB","-4, Внеоборотные активы, RUB","-3, Внеоборотные активы, RUB","-2, Внеоборотные активы, RUB","-1, Внеоборотные активы, RUB","-4, Дебиторская задолженность, RUB","-3, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB","-1, Дебиторская задолженность, RUB","-4, Оборотные активы, RUB","-3, Оборотные активы, RUB","-2, Оборотные активы, RUB","-1, Оборотные активы, RUB","-4, Уставный капитал , RUB","-3, Уставный капитал , RUB","-2, Уставный капитал , RUB","-1, Уставный капитал , RUB","-4, Капитал и резервы, RUB","-3, Капитал и резервы, RUB","-2, Капитал и резервы, RUB","-1, Капитал и резервы, RUB","-4, Заёмные средства (долгосрочные), RUB","-3, Заёмные средства (долгосрочные), RUB","-2, Заёмные средства (долгосрочные), RUB","-1, Заёмные средства (долгосрочные), RUB","-4, Долгосрочные обязательства, RUB","-3, Долгосрочные обязательства, RUB","-2, Долгосрочные обязательства, RUB","-1, Долгосрочные обязательства, RUB","-4, Заёмные средства (краткосрочные), RUB","-3, Заёмные средства (краткосрочные), RUB","-2, Заёмные средства (краткосрочные), RUB","-1, Заёмные средства (краткосрочные), RUB","-4, Кредиторская задолженность, RUB","-3, Кредиторская задолженность, RUB","-2, Кредиторская задолженность, RUB","-1, Кредиторская задолженность, RUB","-4, Краткосрочные обязательства, RUB","-3, Краткосрочные обязательства, RUB","-2, Краткосрочные обязательства, RUB","-1, Краткосрочные обязательства, RUB","-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB","-4, Себестоимость продаж, RUB","-3, Себестоимость продаж, RUB","-2, Себестоимость продаж, RUB","-1, Себестоимость продаж, RUB","-4, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","-4, Прибыль (убыток) от продажи, RUB","-3, Прибыль (убыток) от продажи, RUB","-2, Прибыль (убыток) от продажи, RUB","-1, Прибыль (убыток) от продажи, RUB",year,binary_target,"Макс. ПДЗ за -1 год, дней","Сред. ПДЗ за -1 год, дней","Кол-во просрочек свыше 5-ти дней за -1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за -1 год, руб.","Кол-во раз ПДЗ за -1 год, шт.",Нематериальные активы total mean growth,Основные средства total mean growth,Внеоборотные активы total mean growth,Дебиторская задолженность total mean growth,Оборотные активы total mean growth,Уставный капитал total mean growth,Капитал и резервы total mean growth,Заёмные средства (долгосрочные) total mean growth,Долгосрочные обязательства total mean growth,Заёмные средства (краткосрочные) total mean growth,Кредиторская задолженность total mean growth,Краткосрочные обязательства total mean growth,Выручка total mean growth,Себестоимость продаж total mean growth,Прибыль (убыток) до налогообложения total mean growth,Прибыль (убыток) от продажи total mean growth
2,2,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,30,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,Х,10,Х,Х,Н/Д,Х,Х,Х,Х,Х,Х,40.0,2.468153e+06,1.288025e+07,8.694904e+06,4.958599e+06,9.698064e+08,9.444166e+08,9.287389e+08,1.027508e+09,9.732745e+08,1.036412e+09,1.018682e+09,1.233413e+09,2.814675e+08,2.321070e+08,2.280497e+08,2.770312e+08,1.286917e+09,1.489083e+09,1.590217e+09,8.377650e+08,1.964459e+07,1.964459e+07,1.964459e+07,1.964459e+07,2.110425e+09,2.339440e+09,2.378698e+09,1.854769e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.339809e+07,1.183121e+07,2.843949e+06,0.000000e+00,0.0

In [61]:
df_1[col_name].abs().min()

0.0

In [59]:
df_1[col_name].abs().max()

398315826114.6497

In [60]:
df_1[col_name].max()

398315826114.6497

In [43]:
col_name

'-1, Дебиторская задолженность, RUB'

In [44]:
df_1[col_name].min(skipna=True)

0.0

In [ ]:
df_1

In [11]:
def f(row):
    cols = row.index.values

    return row[[cols[0], cols[1]]]

In [20]:
r = df_1.iloc[2]

In [21]:
f(r)

Наименование ДП    2
Факт. 1            Х
Name: 2, dtype: object

In [14]:
df_1.apply(f, axis=1)

,Наименование ДП,Факт. 1
0,0,Х
1,1,Х
2,2,Х
3,3,Х
4,4,Х
...,...,...
851,522,NaN
852,523,NaN
853,525,NaN
854,527,NaN


In [ ]:
def add_log_values(row, fin_feat_name):
    for i in range(-5, 0):
        col_name = 

In [2]:
df_0 = create_df_0years_known()

In [3]:
df_0

,Наименование ДП,"-3, Нематериальные активы, RUB","-2, Нематериальные активы, RUB","-1, Нематериальные активы, RUB","-3, Основные средства , RUB","-2, Основные средства , RUB","-1, Основные средства , RUB","-3, Внеоборотные активы, RUB","-2, Внеоборотные активы, RUB","-1, Внеоборотные активы, RUB","-3, Дебиторская задолженность, RUB","-2, Дебиторская задолженность, RUB","-1, Дебиторская задолженность, RUB","-3, Оборотные активы, RUB","-2, Оборотные активы, RUB","-1, Оборотные активы, RUB","-3, Уставный капитал , RUB","-2, Уставный капитал , RUB","-1, Уставный капитал , RUB","-3, Капитал и резервы, RUB","-2, Капитал и резервы, RUB","-1, Капитал и резервы, RUB","-3, Заёмные средства (долгосрочные), RUB","-2, Заёмные средства (долгосрочные), RUB","-1, Заёмные средства (долгосрочные), RUB","-3, Долгосрочные обязательства, RUB","-2, Долгосрочные обязательства, RUB","-1, Долгосрочные обязательства, RUB","-3, Заёмные средства (краткосрочные), RUB","-2, Заёмные средства (краткосрочные), RUB","-1, Заёмные средства (краткосрочные), RUB","-3, Кредиторская задолженность, RUB","-2, Кредиторская задолженность, RUB","-1, Кредиторская задолженность, RUB","-3, Краткосрочные обязательства, RUB","-2, Краткосрочные обязательства, RUB","-1, Краткосрочные обязательства, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB","-3, Себестоимость продаж, RUB","-2, Себестоимость продаж, RUB","-1, Себестоимость продаж, RUB","-3, Прибыль (убыток) до налогообложения , RUB","-2, Прибыль (убыток) до налогообложения , RUB","-1, Прибыль (убыток) до налогообложения , RUB","-3, Прибыль (убыток) от продажи, RUB","-2, Прибыль (убыток) от продажи, RUB","-1, Прибыль (убыток) от продажи, RUB",year,binary_target,Итого,"-4, Нематериальные активы, RUB","-4, Основные средства , RUB","-4, Внеоборотные активы, RUB","-4, Дебиторская задолженность, RUB","-4, Оборотные активы, RUB","-4, Уставный капитал , RUB","-4, Капитал и резервы, RUB","-4, Заёмные средства (долгосрочные), RUB","-4, Долгосрочные обязательства, RUB","-4, Заёмные средства (краткосрочные), RUB","-4, Кредиторская задолженность, RUB","-4, Краткосрочные обязательства, RUB","-4, Выручка, RUB","-4, Себестоимость продаж, RUB","-4, Прибыль (убыток) до налогообложения , RUB","-4, Прибыль (убыток) от продажи, RUB",Нематериальные активы total mean growth,Основные средства total mean growth,Внеоборотные активы total mean growth,Дебиторская задолженность total mean growth,Оборотные активы total mean growth,Уставный капитал total mean growth,Капитал и резервы total mean growth,Заёмные средства (долгосрочные) total mean growth,Долгосрочные обязательства total mean growth,Заёмные средства (краткосрочные) total mean growth,Кредиторская задолженность total mean growth,Краткосрочные обязательства total mean growth,Выручка total mean growth,Себестоимость продаж total mean growth,Прибыль (убыток) до налогообложения total mean growth,Прибыль (убыток) от продажи total mean growth
0,0,2.895541e+06,6.245860e+06,9.050955e+06,1.889724e+09,1.803724e+09,2.145239e+09,1.928685e+09,1.880842e+09,2.440078e+09,3.583766e+09,3.683359e+09,3.475461e+09,5.159380e+09,5.897326e+09,6.077448e+09,6.294000e+08,6.294000e+08,6.294000e+08,2.673020e+09,2.971017e+09,2.289274e+09,0.000000e+00,0.000000e+00,0.000000e+00,2.498777e+08,2.589732e+08,2.712210e+08,0.000000e+00,0.000000e+00,3.923917e+08,4.080056e+09,4.452922e+09,5.476906e+09,4.165168e+09,4.548178e+09,5.957031e+09,2.667698e+10,4.162693e+10,3.703952e+10,2.156814e+10,3.335443e+10,3.438313e+10,3.179697e+09,6.167667e+09,7.461828e+08,3.280355e+09,6.200120e+09,8.716191e+08,2019,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768000,0.065464,0.124789,-0.015226,0.085330,0.000000,-0.074561,0.067398,0.041833,0.092383,0.158603,0.195910,0.178323,0.262602,-0.515571,-0.484531
1,1,0.000000e+00,3.885350e+04,3.439490e+04,1.448108e+08,1.487153e+08,1.526427e+08,1.456783e+08,1.497987e+08,1.538924e+08,5.584777e+07,4.678217e+07,1.563478e+08,1.489153e+08,1.898987e+08,2.914637e+08,1.345860e+

In [4]:
df_0['Себестоимость продаж total mean growth'].median()

0.11576440190461379

In [13]:
row = df_0.iloc[0]

In [20]:
total_growth(row, fin_feat_name="Себестоимость продаж")

-3 -1 21568144585.98726 34383128662.42038


0.2626015062032898

In [17]:
df_0.apply(partial(total_growth, fin_feat_name="Себестоимость продаж"), axis=1)

-4 -1 nan 34383128662.42038
-4 -1 nan 416979617.8343949
-4 -1 nan 2316724203.821656
-4 -1 nan 11713152229.29936
-4 -1 nan 26525468789.80892
-4 -1 nan 14396717834.3949
-4 -1 nan 49694920382.1656
-4 -1 nan 366075796.178344
-4 -1 nan 922115923.5668788
-4 -1 nan 289257324.8407643
-4 -1 nan 189451666242.0382
-4 -1 nan 615086624.2038217
-4 -1 nan 669937579.6178343
-4 -1 nan 1507584076.433121
-4 -1 nan 701822292.9936305
-4 -1 nan 904256687.8980892
-4 -1 nan 331794904.4585987
-4 -1 nan 1561664331.210191
-4 -1 nan 590376433.1210191
-4 -1 nan 1436126751.592357
-4 -1 nan 352445222.9299363
-4 -1 nan 478896815.2866242
-4 -1 nan 6092356.687898089
-4 -1 nan 5213312101.910828
-4 -1 nan 3344158598.726115
-4 -1 nan 23935845859.87261
-4 -1 nan 337096178.343949
-4 -1 nan 16695137579.61783
-4 -1 nan 211112738853.5032
-4 -1 nan 6478064331.210191
-4 -1 nan 74991734394.90445
-4 -1 nan 2330573248.407644
-4 -1 nan 317154140.1273885
-4 -1 nan 287241401.2738853
-4 -1 nan 811074522.2929935
-4 -1 nan 380916560.5095

-4 -1 162821019.1082802 497012101.910828
-4 -1 481780254.7770701 586657324.8407643
-4 -1 42608917.19745223 12021656.05095541
-4 -1 6280000636.942675 25018059235.66879
-4 -1 50903352866.24204 69296390445.85986
-4 -1 168587261.1464968 213957961.7834395
-4 -1 58188535.03184713 67679617.8343949
-4 -1 119929299.3630573 170905732.4840764
-4 -1 86422929.93630573 167326114.6496815
-4 -1 19839219108.28025 23382282165.60509
-4 -1 332271337.5796178 4272947133.757962
-4 -1 44753503.18471337 119304458.5987261
-4 -1 619522292.9936305 1939520382.165605
-4 -1 196528662.4203822 227484713.3757962
-2 -1 164061783.4394904 203412738.8535032
-4 -1 85727388.53503184 105894904.4585987
-2 -1 319617834.3949044 548750318.4713376
-4 -1 168156050.955414 180814649.6815286
-2 -1 515637579.6178344 726750318.4713376
-4 -1 3243728025.477707 2411076433.121019
-4 -1 862508917.1974522 1341455414.012739
-4 -1 190157324.8407643 284204458.5987261
-4 -1 2543949.044585987 32949681.52866242
-4 -1 222632484.0764331 331896178.343

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
1382    1.268473
1383         NaN
1384   -0.379301
1385   -0.074728
1386         NaN
Length: 1387, dtype: float64

In [19]:
def total_growth(row, fin_feat_name):
    min_year, min_year_val = None, None
    max_year, max_year_val = None, None
    for col in row.index.values:
        
        if fin_feat_name not in col:
            continue
        
        value = row[col]
        if (value < 10) or np.isnan(value):
            continue
        
        try:
            year = int(col[:2])
        except:
            print(col)
            continue
        if (min_year is None) or year < min_year:
            min_year = year
            min_year_val = value
        if (max_year is None) or year > max_year:
            max_year = year
            max_year_val = value
    
    if (min_year is None) or (min_year == max_year):
        return np.nan

    print(min_year, max_year, min_year_val, max_year_val)
    #delta_val = max_year_val / min_year_val
    #print(delta_val)
    return (max_year_val / min_year_val) ** (1.0 / (max_year-min_year)) - 1

In [2]:
df_0 = create_df_0years_known()

In [17]:
list(range(-4 ,0))

[-4, -3, -2, -1]

In [47]:
df = df_0[df_0['year'] == '2020'][['Наименование ДП', *[str(x) + ', Выручка, RUB' for x in range(-4, 0)]]]

In [48]:
df.head(4)

,Наименование ДП,"-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB"
531,0,2.667698e+10,4.162693e+10,3.703952e+10,3.234755e+10
532,1,2.554898e+08,2.573669e+08,5.434306e+08,7.241261e+08
533,2,2.786577e+09,2.699397e+09,2.592420e+09,2.754208e+09
534,3,1.689520e+10,1.778944e+10,2.165799e+10,1.880440e+10


In [24]:
(df['-1, Выручка, RUB'] < 10).sum()

10

In [25]:
(df['-1, Выручка, RUB'] >= 10).sum()

521

In [26]:
df.shape

(531, 5)

In [27]:
df[df['-1, Выручка, RUB'] < 10]

,Наименование ДП,"-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB"
583,52,3.185758e+09,3.348076e+09,3.025222e+09,0.0
588,57,2.861519e+10,1.725111e+10,0.000000e+00,0.0
591,60,6.760121e+09,4.734053e+09,1.104855e+10,0.0
600,69,0.000000e+00,0.000000e+00,0.000000e+00,0.0
871,340,0.000000e+00,0.000000e+00,0.000000e+00,0.0
986,455,0.000000e+00,1.326115e+06,7.851592e+06,0.0
1019,488,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1051,520,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1054,523,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1059,528,0.000000e+00,0.000000e+00,0.000000e+00,0.0


In [28]:
fin_feat_name = ', Выручка, RUB'

In [29]:
for i, row in df.iterrows():
    break

In [34]:
row

Наименование ДП     0.000000e+00
-4, Выручка, RUB    2.667698e+10
-3, Выручка, RUB    4.162693e+10
-2, Выручка, RUB    3.703952e+10
-1, Выручка, RUB    3.234755e+10
Name: 531, dtype: float64

In [36]:
row.index.values

array(['Наименование ДП', '-4, Выручка, RUB', '-3, Выручка, RUB',
       '-2, Выручка, RUB', '-1, Выручка, RUB'], dtype=object)

In [38]:
fin_feat_name in '-4, Выручка, RUB'

True

In [ ]:
def add_growth(df):
    

In [55]:
row

Наименование ДП     0.000000e+00
-4, Выручка, RUB    2.667698e+10
-3, Выручка, RUB    4.162693e+10
-2, Выручка, RUB    3.703952e+10
-1, Выручка, RUB    3.234755e+10
Name: 531, dtype: float64

In [64]:
f(row, fin_feat_name)

-4 -1 26676980254.77707 32347554140.12739
1.2125643094230985


0.06635446977110404

In [74]:
df['growth'].mean(skipna=True)

0.5607566532127559

In [75]:
df['growth'].median(skipna=True)

0.12131261507049595

In [76]:
df['growth'].std(skipna=True)

2.180733690018153

In [54]:
df

,Наименование ДП,"-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB"
531,0,2.667698e+10,4.162693e+10,3.703952e+10,3.234755e+10
532,1,2.554898e+08,2.573669e+08,5.434306e+08,7.241261e+08
533,2,2.786577e+09,2.699397e+09,2.592420e+09,2.754208e+09
534,3,1.689520e+10,1.778944e+10,2.165799e+10,1.880440e+10
535,4,4.716934e+10,6.658243e+10,8.611910e+10,9.409765e+10
...,...,...,...,...,...
1057,526,8.098535e+07,5.350955e+07,1.154089e+08,3.135331e+08
1058,527,1.437658e+09,1.328392e+09,1.188227e+09,1.106208e+09
1059,528,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1060,529,1.058599e+07,1.725166e+08,4.278682e+08,7.467573e+08


In [68]:
pd.isna(df.apply(partial(f, fin_feat_name=fin_feat_name), axis=1)).sum()

22

In [69]:
df['growth'] = df.apply(partial(f, fin_feat_name=fin_feat_name), axis=1)

In [79]:
df['growth'].isna()

531     False
532     False
533     False
534     False
535     False
        ...  
1057    False
1058    False
1059     True
1060    False
1061    False
Name: growth, Length: 531, dtype: bool

In [81]:
df.loc[df['growth'].isna(), :]

,Наименование ДП,"-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB",growth
600,69,0.0,0.0,0.0,0.000000e+00,NaN
749,218,0.0,0.0,0.0,1.985166e+08,NaN
750,219,0.0,0.0,0.0,5.623312e+07,NaN
752,221,0.0,0.0,0.0,6.156115e+07,NaN
775,244,0.0,0.0,0.0,6.170701e+07,NaN
847,316,0.0,0.0,0.0,4.757643e+07,NaN
853,322,0.0,0.0,0.0,3.443943e+08,NaN
871,340,0.0,0.0,0.0,0.000000e+00,NaN
991,460,0.0,0.0,0.0,3.849561e+08,NaN
997,466,0.0,0.0,0.0,7.205478e+07,NaN


In [70]:
df

,Наименование ДП,"-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB",growth
531,0,2.667698e+10,4.162693e+10,3.703952e+10,3.234755e+10,0.066354
532,1,2.554898e+08,2.573669e+08,5.434306e+08,7.241261e+08,0.415186
533,2,2.786577e+09,2.699397e+09,2.592420e+09,2.754208e+09,-0.003887
534,3,1.689520e+10,1.778944e+10,2.165799e+10,1.880440e+10,0.036331
535,4,4.716934e+10,6.658243e+10,8.611910e+10,9.409765e+10,0.258847
...,...,...,...,...,...,...
1057,526,8.098535e+07,5.350955e+07,1.154089e+08,3.135331e+08,0.570215
1058,527,1.437658e+09,1.328392e+09,1.188227e+09,1.106208e+09,-0.083652
1059,528,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
1060,529,1.058599e+07,1.725166e+08,4.278682e+08,7.467573e+08,3.131896


In [71]:
df[df['growth'].isna()]

,Наименование ДП,"-4, Выручка, RUB","-3, Выручка, RUB","-2, Выручка, RUB","-1, Выручка, RUB",growth
600,69,0.0,0.0,0.0,0.000000e+00,NaN
749,218,0.0,0.0,0.0,1.985166e+08,NaN
750,219,0.0,0.0,0.0,5.623312e+07,NaN
752,221,0.0,0.0,0.0,6.156115e+07,NaN
775,244,0.0,0.0,0.0,6.170701e+07,NaN
847,316,0.0,0.0,0.0,4.757643e+07,NaN
853,322,0.0,0.0,0.0,3.443943e+08,NaN
871,340,0.0,0.0,0.0,0.000000e+00,NaN
991,460,0.0,0.0,0.0,3.849561e+08,NaN
997,466,0.0,0.0,0.0,7.205478e+07,NaN
